<a href="https://colab.research.google.com/github/BinXBioLab/Data_Science_Capstone_Fall_2022/blob/main/CCI_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook Purpose:

This notebook should be used after CellPhoneDB has been run and output files have been saved per grouping. The gain_loss_analysis() function can be used to calculate gain, loss, or no change in cell cell interactions using the pvalue.txt file output from CPDB. 

### Functions:
  - gain_loss_analysis(pvalue_control, pvalue_case, filepath, save_as) 
  - plot_gain() -- *in progress*
  - plot_loss() -- *in progress*


In [ ]:
#create function that does analysis in one run 

def gain_loss_analysis(pvalue_control, pvalue_case, filepath, save_as):
  """
  Analyzes pvalue changes between 2 cases (i.e. Control vs. Mutant) to determine cell cell 
  interactions that produced a gain, a loss, or no change between cases. 

  Input: 
  - pvalue_control: string (file name) generated by CellPhoneDB statistical_analysis method 
    on control data, .txt file
  - pvalue_case: string (file name) generated by CellPhoneDB statistical_analysis 
    method on case/mutant/disease data, .txt file
  - filepath: string (filepath) where pvalue files can be found & where 
    output files will be saved
  - save_as: string (filename) how you'd like the results file to be named 
    (i.e. 'results70d' or 'results150d')

  Returns: results.csv, loss_df.csv, gain_df.csv in filepath location using name defined by user
  """

  #load files 
  pvalues_control = pd.read_csv(os.path.join(filepath, pvalue_control), sep='\t')
  pvalues_case = pd.read_csv(os.path.join(filepath, pvalue_case), sep='\t')

  #make list of cell|cell's in dataset
  cc_list = list(pvalues_case.columns[11:])
  #check id_interactions that are in both case and control
  id_interactions = set(list(pvalues_case['id_cp_interaction'])).intersection(list(pvalues_control['id_cp_interaction']))
  #filter for intersection only
  control_filter = pvalues_control[pvalues_control['id_cp_interaction'].isin(id_interactions)]
  case_filter = pvalues_case[pvalues_case['id_cp_interaction'].isin(id_interactions)]

  #add two columns to count total gain and loss per row
  cc_list.append('total_gain')
  cc_list.append('total_loss')
  
  #initialize dataframe
  results_df = pd.DataFrame(columns=(cc_list)) #results df has cellx | celly as column headers

  #loop through each row of control/case filtered dataframe
  for i in range(len(control_filter)): 
    control_row = control_filter.iloc[i].to_numpy()[11:] # values from control
    case_row = case_filter.iloc[i].to_numpy()[11:] # values from case 
    result = [] # reset result list for each new row
    gain = 0 # reset gain count for each new row
    loss = 0 # reset loss count for each new row

    #loop through each value in both lists to compare
    for j in range(len(control_row)):
      if control_row[j] > 0.5 and case_row[j] < 0.05:
        result.append('Gain')
        gain+=1
      elif control_row[j] < 0.05 and case_row[j] > 0.5:
        result.append('Loss')
        loss+=1
      else: 
        result.append('No Change')
    result.append(gain) #add gain count to end of results list
    result.append(loss) #add loss count to end of results list
    
    #add list of results to dataframe at bottom
    results_df.loc[len(results_df)] = result

  #merge initial columns back with results_df on index 
  header_columns = case_filter[['id_cp_interaction', 'interacting_pair', 'partner_a', 'partner_b', 'gene_a', 'gene_b',
                          'secreted', 'receptor_a', 'receptor_b', 'annotation_strategy', 'is_integrin']]

  results = header_columns.join(results_df)

  results.reset_index(inplace=True)
  results.drop(columns='index', inplace=True)
  
  #create gain and loss tables
  loss_df = results[results['total_loss'] > 0]
  gain_df = results[results['total_gain'] > 0]

  save_results = f'{save_as}.csv'
  save_loss = f'{save_as}_LOSS.csv'
  save_gain = f'{save_as}_GAIN.csv'

  #save dataframes
  results.to_csv(os.path.join(filepath, save_results), index=False)
  loss_df.to_csv(os.path.join(filepath, save_loss), index=False)
  gain_df.to_csv(os.path.join(filepath, save_gain), index=False)

In [ ]:
def plot_gain(gains_df, filepath, save_as):
  """
  Plots id's that have at least a user defined level of gains across cell|cell interactions as heatplot

  Input: 
  - gains_df: string (filename)
  - filepath: string (filepath)
  - save_as: string (filename)

  Output: 
  - Heatplot saved to current directory 
  - Plots heatplot in notebook 
  """

  

In [ ]:
def plot_loss():
  """

  """

### Code to run Analysis 

In [ ]:
import pandas as pd 
import os 
import numpy as np
from google.colab import drive
drive.mount('/mntDrive')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 50)

Update your current directory to load pvalues.txt files appropriately according to your mounted drive filepath. 

Example syntax:

`%cd /mntDrive/MyDrive/Capstone/CellphoneDB_Files/22qAGGR01/`

In [ ]:
%cd /mntDrive/MyDrive/Folder1/Folder2/Folderwithoutput/

#### Run Analysis
example syntax:



```
gain_loss_analysis('CONTROL150_out/pvalues.txt', 'CASE150_out/pvalues.txt', '/mntDrive/MyDrive/Capstone/CellphoneDB_Files/22qAGGR01/', 'Results150')
```



In [ ]:
#Run analysis 
gain_loss_analysis(pvalue_control, pvalue_case, filepath, save_as)

In [ ]:
#plot gains
plot_gain()

In [ ]:
#plot losses
plot_loss()